In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import os

from keras.optimizers import Adam

from fully_conv_model_for_lidar import fcn_model
from util_func import *
from train import *

from keras.utils.generic_utils import get_custom_objects
#loss = SSD_Loss(neg_pos_ratio=neg_pos_ratio, alpha=alpha)
get_custom_objects().update({"my_loss": my_loss})

#import keras.losses
#keras.losses.custom_loss = my_loss

Using TensorFlow backend.


In [2]:
model = load_model('saved_model/model_2_one_frame_with_aug.h5')

[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 64, 256, 8) dtype=float32>, None, None]


In [3]:
#n = np.random.randint(154)
n = 0
print(n)
lidar = np.load('./extract_kiti/0005/lidar/lidar_'+ str(n) + '.npy')
gt_box = np.load('./extract_kiti/0005/gt_boxes3d/gt_boxes3d_'+ str(n) + '.npy')
#view, box = cylindrical_projection_for_training(lidar, gt_box)
#print(view.shape, box.shape)

0


In [4]:
offset_range = 5*np.pi/180
flip = 0
offset = np.random.uniform(low=-offset_range, high=offset_range)
#offset = -5*np.pi/180
#offset = 0
print(offset)
lidar, gt_box = augmentation(offset, flip, lidar, gt_box)

-0.00358375819568


In [5]:
viz_mayavi_with_labels(lidar, gt_box)

In [6]:
all_boxes, cluster_boxes = predict_boxes(model,lidar, cluster = True,
                                         seg_thres=0.5, cluster_dist = 0.2, min_dist = 1.5, neigbor_thres = 3)
print(len(all_boxes))
print(len(cluster_boxes))

1088
3


In [7]:
n_boxes = len(all_boxes)
choosen_boxes = np.random.randint(n_boxes, size = 30)
print(choosen_boxes)
viz_mayavi_with_labels(lidar, all_boxes[choosen_boxes])

[ 951  380   70  313  898  540  664  958 1070  615  874 1052  139  968  812
  898  329  398  752  653  214  825  436  867 1023  451  516  720  803  144]


In [8]:
#print(len(cluster_boxes))
viz_mayavi_with_labels(lidar, cluster_boxes)